# Propagation du COVID-19 en France 
## - Préparation des données -
`AROUI Khaoula`
<br><br>
Sommaire :
 - <a href="#p1"> Importation des bases des données brutes</a>
 - <a href="#p2"> Aperçu sur les bases de données brutes et préparation de chacune</a> 
 - <a href="#p2-1"> 1. Données sur la propagation du covid-19 </a>  
 - <a href="#p2-2"> 2. Données sur la couverture vaccinale </a>  
 - <a href="#p3"> Jointure des deux premières sources</a>
 - <a href="#p3-1"> 1. Vérification des valeurs manquantes et préparation des données d'analyse</a>
 - <a href="#p5"> Jointure avec les données des densités de population</a>

## <a name="#p1"> Importation des bases des données brutes</a>

In [ ]:
# Importation des librairies requises
print(f'Importing required packages ...')
from pyspark.sql import SparkSession 
from pyspark import SparkContext
from pyspark import SparkConf
import pyspark.sql.functions as F # class of spark functions
import pyspark.sql.types as T
from pyspark.sql import Window
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.api.types import CategoricalDtype
import datetime
import sys,os
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.4f}'.format

import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore")


ss = SparkSession.builder.config("spark.driver.host", "localhost").getOrCreate()

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
pd.options.display.float_format = '{:,.4f}'.format

/tmp/ipykernel_223/2067002790.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [68]:
# Données sur la propagation du covid-19
source_1 = ss.read.csv(f'../Data/rawdata/table-indicateurs-open-data-dep-serie.csv',sep=',', header=True, inferSchema=True)
# Données sur la couverture vaccinale par départements
source_2 = ss.read.csv(f'../Data/rawdata/vacsi-dep-2023-01-20-19h00.csv',sep=';', header=True, inferSchema=True)
# Données démographiques de la population 
source_3 = ss.read.csv(f'../Data/rawdata/dep.csv',sep=';', header=True, inferSchema=True)
# Données démographiques de les superficie 
source_4 = ss.read.csv(f'../Data/rawdata/superficie.csv',sep=';', header=True, inferSchema=True)
# Données sur les mesures de 10 avril ( apres 20 jours ou moins de la mise en place de restrictions de 20 jours) 
source_5 = ss.read.csv(f'../Data/rawdata/mesures_10_avril_2021.csv',sep=';', header=True, inferSchema=True)

## <a name="#p2"> Aperçu sur les bases de données brutes et préparation de chacune</a> 

### <a name="p2-1">Données sur la propagation du covid-19</a>
##### Ces données présentent la valeur quotidienne de ces 4 indicateurs au niveau national et départemental depuis le 15 mars 2020.

In [19]:
print(f'{source_1.count()} lignes {len(source_1.columns)} colonnes dans data sources 1')

90395 lignes 15 colonnes dans data sources 1


In [21]:
source_1.limit(5).toPandas()

,extract_date,departement,region,libelle_reg,libelle_dep,tx_incid,R,taux_occupation_sae,tx_pos,tx_incid_couleur,R_couleur,taux_occupation_sae_couleur,tx_pos_couleur,nb_orange,nb_rouge
0,2020-03-19,01,84,Auvergne Rhône Alpes,Ain,NaN,NaN,14.1000,NaN,None,None,vert,None,0,0
1,2020-03-18,01,84,Auvergne Rhône Alpes,Ain,NaN,NaN,6.3000,NaN,None,None,vert,None,0,0
2,2021-01-21,01,84,Auvergne Rhône Alpes,Ain,212.0400,NaN,71.6000,9.9600,rouge,None,rouge,orange,1,2
3,2021-01-22,01,84,Auvergne Rhône Alpes,Ain,212.3400,NaN,73.2000,9.8531,rouge,None,rouge,orange,1,2
4,2021-01-23,01,84,Auvergne Rhône Alpes,Ain,217.0600,NaN,71.4000,9.9616,rouge,None,rouge,orange,1,2


In [22]:
# Garder uniquement les variables d'interêt
columns_list=['extract_date','region', 'libelle_reg', 'departement', 'libelle_dep',
       'tx_incid', 'R', 'taux_occupation_sae', 'tx_pos', 'tx_incid_couleur',
       'R_couleur', 'taux_occupation_sae_couleur', 'tx_pos_couleur',
       'nb_orange', 'nb_rouge']
source_1=source_1.select(columns_list)

In [32]:
print(source_1.groupBy(['region', 'libelle_reg']).count().show())
print(source_1.groupBy(['departement', 'libelle_dep']).count().show())
print(f"la première date est: {source_1.agg({'extract_date': 'min'}).collect()[0][0]}")
print(f"la dernière date est: {source_1.agg({'extract_date': 'max'}).collect()[0][0]}")

+------+--------------------+-----+
|region|         libelle_reg|count|
+------+--------------------+-----+
|    84|Auvergne Rhône Alpes|10740|
|    27|Bourgogne Franche...| 7160|
|    53|            Bretagne| 3580|
|    94|               Corse| 1790|
|    24| Centre Val de Loire| 5370|
|     3|              Guyane|  895|
|     1|          Guadeloupe|  895|
|    44|           Grand Est| 8950|
|    75|  Nouvelle Aquitaine|10740|
|    32|     Hauts de France| 4475|
|    11|       Ile de France| 7160|
|     2|          Martinique|  895|
|     6|             Mayotte|  895|
|    28|           Normandie| 4475|
|    76|           Occitanie|11635|
|     4|          La Réunion|  895|
|    93|Provence Alpes Cô...| 5370|
|    52|       Pays de Loire| 4475|
+------+--------------------+-----+

None
+-----------+-------------+-----+
|departement|  libelle_dep|count|
+-----------+-------------+-----+
|         67|     Bas-Rhin|  895|
|         42|        Loire|  895|
|        973|       Guyane|  895

In [ ]:
#Vérifier les valeurs manquantes pour tous les colonnes
print("\n Valeurs manquantes :")
display(source_1.isnull().sum())
print('-'*20)
print('\n'*2)


 Valeurs manquantes :


extract_date                   0    
region                         0    
libelle_reg                    0    
departement                    0    
libelle_dep                    0    
tx_incid                       16463
R                              79591
taux_occupation_sae            0    
tx_pos                         17069
tx_incid_couleur               16463
R_couleur                      79591
taux_occupation_sae_couleur    0    
tx_pos_couleur                 17069
nb_orange                      0    
nb_rouge                       0    
dtype: int64

--------------------





In [42]:
#Vérifier les valeurs manquantes pour tous les colonnes
print("\n Valeurs manquantes :")
print(source_1.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in source_1.columns]).toPandas().transpose())
print('-'*20)
print('\n'*2)


 Valeurs manquantes :
                                 0
extract_date                     0
region                           0
libelle_reg                      0
departement                      0
libelle_dep                      0
tx_incid                     16463
R                            79591
taux_occupation_sae              0
tx_pos                       17069
tx_incid_couleur             16463
R_couleur                    79591
taux_occupation_sae_couleur      0
tx_pos_couleur               17069
nb_orange                        0
nb_rouge                         0
--------------------





### <a name="p2-2">Données sur la couverture vaccinale</a>
##### Le gouvernement a présenté dans le cadre de la deuxième étape du plan de déconfinement, la carte de synthèse des départements qui sert de référence pour les mesures différenciées appliquées depuis le 2 juin. Cette carte est construite sur la base de 4 indicateurs et est complétée par une analyse de risques. Les indicateurs sont: Activité épidémique (taux d'incidence), Taux de positivité des tests virologiques, Facteur de reproduction du virus (évolution du R0), Tension hospitalière sur la capacité en réanimation

In [48]:
print(f'{source_2.count()} lignes {len(source_2.columns)} colonnes dans data sources 2')

77662 lignes 20 colonnes dans data sources 2


In [49]:
source_2.limit(5).toPandas()

,dep,jour,n_dose1,n_complet,n_rappel,n_2_rappel,n_rappel_biv,n_3_rappel,n_cum_dose1,n_cum_complet,n_cum_rappel,n_cum_2_rappel,n_cum_rappel_biv,n_cum_3_rappel,couv_dose1,couv_complet,couv_rappel,couv_2_rappel,couv_rappel_biv,couv_3_rappel
0,01,2020-12-27,0,0,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,01,2020-12-28,0,0,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,01,2020-12-29,0,0,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,01,2020-12-30,0,0,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,01,2020-12-31,1,0,0,0,0,0,1,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [51]:
# Garder uniquement les variables d'interêt
columns_list=['dep', 'jour', 'n_dose1', 'n_complet', 'n_rappel', 'n_2_rappel',
       'n_rappel_biv', 'n_3_rappel', 'n_cum_dose1', 'n_cum_complet',
       'n_cum_rappel', 'n_cum_2_rappel', 'n_cum_rappel_biv', 'n_cum_3_rappel',
       'couv_dose1', 'couv_complet', 'couv_rappel', 'couv_2_rappel',
       'couv_rappel_biv', 'couv_3_rappel']
source_2 = source_2.select(columns_list)

In [53]:
#Vérifier les valeurs manquantes pour tous les colonnes
print("\n Valeurs manquantes :")
print(source_2.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in source_2.columns]).toPandas().transpose())
print('-'*20)
print('\n'*2)


 Valeurs manquantes :
                  0
dep               0
jour              0
n_dose1           0
n_complet         0
n_rappel          0
n_2_rappel        0
n_rappel_biv      0
n_3_rappel        0
n_cum_dose1       0
n_cum_complet     0
n_cum_rappel      0
n_cum_2_rappel    0
n_cum_rappel_biv  0
n_cum_3_rappel    0
couv_dose1        0
couv_complet      0
couv_rappel       0
couv_2_rappel     0
couv_rappel_biv   0
couv_3_rappel     0
--------------------





### <a name="#p3"> Jointure des deux premières sources</a>

In [54]:
source_1 = source_1.withColumnRenamed('extract_date', 'date')
source_2 = (source_2
            .withColumnRenamed('jour', 'date')
            .withColumnRenamed('dep', 'departement')
           )

In [55]:
data = source_1.join(source_2, on=['date', 'departement'], how='inner')

### <a name="#p3-1"> 1. Vérification des valeurs manquantes et préparation des données d'analyse</a>

In [56]:
#Vérifier les valeurs manquantes pour tous les colonnes
print("\n Valeurs manquantes :")
print('-'*20)
display(data.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in data.columns]).toPandas().transpose())
print('-'*20)
print('\n'*2)
print(data.columns)


 Valeurs manquantes :
--------------------
23/03/01 23:11:12 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,0
date,0
departement,0
region,0
libelle_reg,0
libelle_dep,0
tx_incid,10807
R,55754
taux_occupation_sae,0
tx_pos,10807
tx_incid_couleur,10807


--------------------



['date', 'departement', 'region', 'libelle_reg', 'libelle_dep', 'tx_incid', 'R', 'taux_occupation_sae', 'tx_pos', 'tx_incid_couleur', 'R_couleur', 'taux_occupation_sae_couleur', 'tx_pos_couleur', 'nb_orange', 'nb_rouge', 'n_dose1', 'n_complet', 'n_rappel', 'n_2_rappel', 'n_rappel_biv', 'n_3_rappel', 'n_cum_dose1', 'n_cum_complet', 'n_cum_rappel', 'n_cum_2_rappel', 'n_cum_rappel_biv', 'n_cum_3_rappel', 'couv_dose1', 'couv_complet', 'couv_rappel', 'couv_2_rappel', 'couv_rappel_biv', 'couv_3_rappel']


In [57]:
# Ne garder que les colonnes les plus pertinents pour répondre à la problématique 
columns_list=['date', 'region', 'libelle_reg', 'departement', 'libelle_dep',
       'tx_incid', 'R', 'taux_occupation_sae', 'tx_pos', 'tx_incid_couleur',
       'R_couleur', 'taux_occupation_sae_couleur', 'tx_pos_couleur',
       'nb_orange', 'nb_rouge', 'n_dose1', 'n_complet', 'n_rappel',
       'n_2_rappel', 'n_rappel_biv', 'n_3_rappel', 'n_cum_dose1',
       'n_cum_complet', 'n_cum_rappel', 'n_cum_2_rappel', 'n_cum_rappel_biv',
       'n_cum_3_rappel', 'couv_dose1', 'couv_complet', 'couv_rappel',
       'couv_2_rappel', 'couv_rappel_biv', 'couv_3_rappel']
data = data.select(columns_list)

# Renommer les colonnes
data = (data
        .withColumnRenamed('date', 'date')
        .withColumnRenamed('region', 'region_N')
        .withColumnRenamed('libelle_reg', 'Région')
        .withColumnRenamed('tx_incid', "taux d'incidence")
        .withColumnRenamed('R', "Rouge")
       )
print(data.columns)

['date', 'region_N', 'Région', 'departement', 'libelle_dep', "taux d'incidence", 'Rouge', 'taux_occupation_sae', 'tx_pos', 'tx_incid_couleur', 'R_couleur', 'taux_occupation_sae_couleur', 'tx_pos_couleur', 'nb_orange', 'nb_rouge', 'n_dose1', 'n_complet', 'n_rappel', 'n_2_rappel', 'n_rappel_biv', 'n_3_rappel', 'n_cum_dose1', 'n_cum_complet', 'n_cum_rappel', 'n_cum_2_rappel', 'n_cum_rappel_biv', 'n_cum_3_rappel', 'couv_dose1', 'couv_complet', 'couv_rappel', 'couv_2_rappel', 'couv_rappel_biv', 'couv_3_rappel']


In [62]:
print(f"la première date est: {data.agg({'date': 'min'}).collect()[0][0]}")
print(f"la dernière date est: {data.agg({'date': 'max'}).collect()[0][0]}")
print(f'{data.count()} lignes {len(data.columns)} colonnes dans data')

la première date est: 2020-12-27 00:00:00
la dernière date est: 2022-08-29 00:00:00
61711 lignes 33 colonnes dans data


In [61]:
data.limit(5).toPandas()

,date,region_N,Région,departement,libelle_dep,taux d'incidence,Rouge,taux_occupation_sae,tx_pos,tx_incid_couleur,R_couleur,taux_occupation_sae_couleur,tx_pos_couleur,nb_orange,nb_rouge,n_dose1,n_complet,n_rappel,n_2_rappel,n_rappel_biv,n_3_rappel,n_cum_dose1,n_cum_complet,n_cum_rappel,n_cum_2_rappel,n_cum_rappel_biv,n_cum_3_rappel,couv_dose1,couv_complet,couv_rappel,couv_2_rappel,couv_rappel_biv,couv_3_rappel
0,2021-01-21,84,Auvergne Rhône Alpes,01,Ain,212.0400,NaN,71.6000,9.9600,rouge,None,rouge,orange,1,2,1238,7,0,0,0,0,7378,27,0,0,0,0,1.1000,0.0000,0.0000,0.0000,0.0000,0.0000
1,2021-01-22,84,Auvergne Rhône Alpes,01,Ain,212.3400,NaN,73.2000,9.8531,rouge,None,rouge,orange,1,2,839,4,0,0,0,0,8217,31,0,0,0,0,1.2000,0.0000,0.0000,0.0000,0.0000,0.0000
2,2021-01-23,84,Auvergne Rhône Alpes,01,Ain,217.0600,NaN,71.4000,9.9616,rouge,None,rouge,orange,1,2,285,2,0,0,0,0,8502,33,0,0,0,0,1.3000,0.0000,0.0000,0.0000,0.0000,0.0000
3,2021-01-24,84,Auvergne Rhône Alpes,01,Ain,218.7400,NaN,73.2000,10.0293,rouge,None,rouge,rouge,0,3,73,2,0,0,0,0,8575,35,0,0,0,0,1.3000,0.0000,0.0000,0.0000,0.0000,0.0000
4,2021-01-25,84,Auvergne Rhône Alpes,01,Ain,225.1300,NaN,75.7000,10.0476,rouge,None,rouge,rouge,0,3,633,2,0,0,0,0,9208,37,0,0,0,0,1.4000,0.0000,0.0000,0.0000,0.0000,0.0000


### <a name="#pextract"> Exctraction de data en tant que données distribuées 'parquet' vers le domaine de stockage</a>

In [64]:
print(f'Exporting Data ...')
(data
 .write.mode('overwrite')
 .parquet(f"../Data/rawdata/data.parquet")
)

Exporting Data ...


### <a name="#pimport"> Re-importation de données distribuées 'parquet'</a>

In [71]:
data = (ss
        .read
        .parquet(f"../Data/rawdata/data.parquet")
       )

### <a name="#p5"> Jointure avec les données sur la population le sexe et l'age de la population</a>

In [76]:
print(source_3.columns)
# faire la jointure avec la source 3
data2= data.join(source_3, on=['departement', 'libelle_dep'], how='inner')
print(f'{data2.count()} lignes {len(data2.columns)} colonnes dans data')

['departement', 'libelle_dep', 'Ensemble', 'Part des femmes (en %)', 'Part des hommes (en %)', 'Part des 0 � 24 ans de la population (en %)', 'Part des 25 � 59 ans (en %)', 'Part des 60 ans ou plus (en %)', 'dont part des 75 ans ou plus (en %)']
45825 lignes 40 colonnes dans data


In [77]:
data2.limit(5).toPandas()

,departement,libelle_dep,date,region_N,Région,taux d'incidence,Rouge,taux_occupation_sae,tx_pos,tx_incid_couleur,R_couleur,taux_occupation_sae_couleur,tx_pos_couleur,nb_orange,nb_rouge,n_dose1,n_complet,n_rappel,n_2_rappel,n_rappel_biv,n_3_rappel,n_cum_dose1,n_cum_complet,n_cum_rappel,n_cum_2_rappel,n_cum_rappel_biv,n_cum_3_rappel,couv_dose1,couv_complet,couv_rappel,couv_2_rappel,couv_rappel_biv,couv_3_rappel,Ensemble,Part des femmes (en %),Part des hommes (en %),Part des 0 � 24 ans de la population (en %),Part des 25 � 59 ans (en %),Part des 60 ans ou plus (en %),dont part des 75 ans ou plus (en %)
0,02,Aisne,2021-03-18,32,Hauts de France,295.0300,1.0800,116.4000,8.2769,rouge,orange,rouge,orange,2,2,694,626,0,0,0,0,43506,18007,0,0,0,0,8.3000,3.4000,0.0000,0.0000,0.0000,0.0000,525.503,"51,3","48,7","29,6","42,1","28,4","9,3"
1,02,Aisne,2021-03-19,32,Hauts de France,307.2000,NaN,118.3000,8.3342,rouge,None,rouge,orange,1,2,1243,303,0,0,0,0,44749,18310,0,0,0,0,8.5000,3.5000,0.0000,0.0000,0.0000,0.0000,525.503,"51,3","48,7","29,6","42,1","28,4","9,3"
2,02,Aisne,2021-03-20,32,Hauts de France,322.0200,NaN,120.8000,8.5716,rouge,None,rouge,orange,1,2,2471,165,0,0,0,0,47220,18475,0,0,0,0,9.0000,3.5000,0.0000,0.0000,0.0000,0.0000,525.503,"51,3","48,7","29,6","42,1","28,4","9,3"
3,02,Aisne,2021-03-21,32,Hauts de France,321.8300,NaN,122.6000,8.5423,rouge,None,rouge,orange,1,2,1405,76,0,0,0,0,48625,18551,0,0,0,0,9.2000,3.5000,0.0000,0.0000,0.0000,0.0000,525.503,"51,3","48,7","29,6","42,1","28,4","9,3"
4,02,Aisne,2021-03-22,32,Hauts de France,346.1600,NaN,130.4000,8.8743,rouge,None,rouge,orange,1,2,802,497,0,0,0,0,49427,19048,0,0,0,0,9.4000,3.6000,0.0000,0.0000,0.0000,0.0000,525.503,"51,3","48,7","29,6","42,1","28,4","9,3"


### <a name="#p5"> Jointure avec les données sur la superficie des départements</a>

In [119]:
print(source_4.columns)
# faire la jointure avec la source 3
data3 = data2.join(source_4, on=['departement','libelle_dep'], how='inner')
print(f'{data3.count()} lignes {len(data3.columns)} colonnes dans data')

['departement', 'libelle_dep', 'Superficie']
45825 lignes 41 colonnes dans data


In [120]:
data3.limit(5).toPandas()

,departement,libelle_dep,date,region_N,Région,taux d'incidence,Rouge,taux_occupation_sae,tx_pos,tx_incid_couleur,R_couleur,taux_occupation_sae_couleur,tx_pos_couleur,nb_orange,nb_rouge,n_dose1,n_complet,n_rappel,n_2_rappel,n_rappel_biv,n_3_rappel,n_cum_dose1,n_cum_complet,n_cum_rappel,n_cum_2_rappel,n_cum_rappel_biv,n_cum_3_rappel,couv_dose1,couv_complet,couv_rappel,couv_2_rappel,couv_rappel_biv,couv_3_rappel,Ensemble,Part des femmes (en %),Part des hommes (en %),Part des 0 � 24 ans de la population (en %),Part des 25 � 59 ans (en %),Part des 60 ans ou plus (en %),dont part des 75 ans ou plus (en %),Superficie
0,02,Aisne,2021-03-18,32,Hauts de France,295.0300,1.0800,116.4000,8.2769,rouge,orange,rouge,orange,2,2,694,626,0,0,0,0,43506,18007,0,0,0,0,8.3000,3.4000,0.0000,0.0000,0.0000,0.0000,525.503,"51,3","48,7","29,6","42,1","28,4","9,3",7 361.7
1,02,Aisne,2021-03-19,32,Hauts de France,307.2000,NaN,118.3000,8.3342,rouge,None,rouge,orange,1,2,1243,303,0,0,0,0,44749,18310,0,0,0,0,8.5000,3.5000,0.0000,0.0000,0.0000,0.0000,525.503,"51,3","48,7","29,6","42,1","28,4","9,3",7 361.7
2,02,Aisne,2021-03-20,32,Hauts de France,322.0200,NaN,120.8000,8.5716,rouge,None,rouge,orange,1,2,2471,165,0,0,0,0,47220,18475,0,0,0,0,9.0000,3.5000,0.0000,0.0000,0.0000,0.0000,525.503,"51,3","48,7","29,6","42,1","28,4","9,3",7 361.7
3,02,Aisne,2021-03-21,32,Hauts de France,321.8300,NaN,122.6000,8.5423,rouge,None,rouge,orange,1,2,1405,76,0,0,0,0,48625,18551,0,0,0,0,9.2000,3.5000,0.0000,0.0000,0.0000,0.0000,525.503,"51,3","48,7","29,6","42,1","28,4","9,3",7 361.7
4,02,Aisne,2021-03-22,32,Hauts de France,346.1600,NaN,130.4000,8.8743,rouge,None,rouge,orange,1,2,802,497,0,0,0,0,49427,19048,0,0,0,0,9.4000,3.6000,0.0000,0.0000,0.0000,0.0000,525.503,"51,3","48,7","29,6","42,1","28,4","9,3",7 361.7


In [122]:
data3.limit(5).toPandas()

,departement,libelle_dep,date,region_N,Région,taux d'incidence,Rouge,taux_occupation_sae,tx_pos,tx_incid_couleur,R_couleur,taux_occupation_sae_couleur,tx_pos_couleur,nb_orange,nb_rouge,n_dose1,n_complet,n_rappel,n_2_rappel,n_rappel_biv,n_3_rappel,n_cum_dose1,n_cum_complet,n_cum_rappel,n_cum_2_rappel,n_cum_rappel_biv,n_cum_3_rappel,couv_dose1,couv_complet,couv_rappel,couv_2_rappel,couv_rappel_biv,couv_3_rappel,Ensemble,Part des femmes (en %),Part des hommes (en %),Part des 0 � 24 ans de la population (en %),Part des 25 � 59 ans (en %),Part des 60 ans ou plus (en %),dont part des 75 ans ou plus (en %),Superficie
0,02,Aisne,2021-03-18,32,Hauts de France,295.0300,1.0800,116.4000,8.2769,rouge,orange,rouge,orange,2,2,694,626,0,0,0,0,43506,18007,0,0,0,0,8.3000,3.4000,0.0000,0.0000,0.0000,0.0000,525.503,"51,3","48,7","29,6","42,1","28,4","9,3",7361.7
1,02,Aisne,2021-03-19,32,Hauts de France,307.2000,NaN,118.3000,8.3342,rouge,None,rouge,orange,1,2,1243,303,0,0,0,0,44749,18310,0,0,0,0,8.5000,3.5000,0.0000,0.0000,0.0000,0.0000,525.503,"51,3","48,7","29,6","42,1","28,4","9,3",7361.7
2,02,Aisne,2021-03-20,32,Hauts de France,322.0200,NaN,120.8000,8.5716,rouge,None,rouge,orange,1,2,2471,165,0,0,0,0,47220,18475,0,0,0,0,9.0000,3.5000,0.0000,0.0000,0.0000,0.0000,525.503,"51,3","48,7","29,6","42,1","28,4","9,3",7361.7
3,02,Aisne,2021-03-21,32,Hauts de France,321.8300,NaN,122.6000,8.5423,rouge,None,rouge,orange,1,2,1405,76,0,0,0,0,48625,18551,0,0,0,0,9.2000,3.5000,0.0000,0.0000,0.0000,0.0000,525.503,"51,3","48,7","29,6","42,1","28,4","9,3",7361.7
4,02,Aisne,2021-03-22,32,Hauts de France,346.1600,NaN,130.4000,8.8743,rouge,None,rouge,orange,1,2,802,497,0,0,0,0,49427,19048,0,0,0,0,9.4000,3.6000,0.0000,0.0000,0.0000,0.0000,525.503,"51,3","48,7","29,6","42,1","28,4","9,3",7361.7


#### Création de la variable densité de population

In [123]:
data3 = (data3
         .withColumn('Superficie', F.regexp_replace(F.col("Superficie"), " ", ""))
         .withColumn('Superficie',F.col('Superficie').cast('double'))
         .withColumn('Ensemble',F.col('Ensemble').cast('double'))
         .withColumn("Densité", F.col('Ensemble') / F.col('Superficie'))
        )

In [124]:
#Vérifier les valeurs manquantes pour tous les colonnes
print("\n Valeurs manquantes :")
print('-'*20)
display(data3.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in data3.columns]).toPandas().transpose())
print('-'*20)
print('\n'*2)
print(data3.columns)


 Valeurs manquantes :
--------------------


,0
departement,0
libelle_dep,0
date,0
region_N,0
Région,0
taux d'incidence,8025
Rouge,41400
taux_occupation_sae,0
tx_pos,8025
tx_incid_couleur,8025


--------------------



['departement', 'libelle_dep', 'date', 'region_N', 'Région', "taux d'incidence", 'Rouge', 'taux_occupation_sae', 'tx_pos', 'tx_incid_couleur', 'R_couleur', 'taux_occupation_sae_couleur', 'tx_pos_couleur', 'nb_orange', 'nb_rouge', 'n_dose1', 'n_complet', 'n_rappel', 'n_2_rappel', 'n_rappel_biv', 'n_3_rappel', 'n_cum_dose1', 'n_cum_complet', 'n_cum_rappel', 'n_cum_2_rappel', 'n_cum_rappel_biv', 'n_cum_3_rappel', 'couv_dose1', 'couv_complet', 'couv_rappel', 'couv_2_rappel', 'couv_rappel_biv', 'couv_3_rappel', 'Ensemble', 'Part des femmes (en %)', 'Part des hommes (en %)', 'Part des 0 � 24 ans de la population (en %)', 'Part des 25 � 59 ans (en %)', 'Part des 60 ans ou plus (en %)', 'dont part des 75 ans ou plus (en %)', 'Superficie', 'Densité']


### jointure avec les Données sur les mesures de 10 avril

In [125]:
print(source_5.columns)
# faire la jointure avec la source 3
data4 = data3.join(source_5, on=['departement','libelle_dep'], how='inner')
print(f'{data4.count()} lignes {len(data4.columns)} colonnes dans data')

['departement', 'libelle_dep', 'Restrictions sanitaires renforc�es']
45825 lignes 43 colonnes dans data


In [126]:
#Vérifier les valeurs manquantes pour tous les colonnes
print("\n Valeurs manquantes :")
print('-'*20)
display(data4.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in data4.columns]).toPandas().transpose())
print('-'*20)
print('\n'*2)
print(data4.columns)


 Valeurs manquantes :
--------------------


,0
departement,0
libelle_dep,0
date,0
region_N,0
Région,0
taux d'incidence,8025
Rouge,41400
taux_occupation_sae,0
tx_pos,8025
tx_incid_couleur,8025


--------------------



['departement', 'libelle_dep', 'date', 'region_N', 'Région', "taux d'incidence", 'Rouge', 'taux_occupation_sae', 'tx_pos', 'tx_incid_couleur', 'R_couleur', 'taux_occupation_sae_couleur', 'tx_pos_couleur', 'nb_orange', 'nb_rouge', 'n_dose1', 'n_complet', 'n_rappel', 'n_2_rappel', 'n_rappel_biv', 'n_3_rappel', 'n_cum_dose1', 'n_cum_complet', 'n_cum_rappel', 'n_cum_2_rappel', 'n_cum_rappel_biv', 'n_cum_3_rappel', 'couv_dose1', 'couv_complet', 'couv_rappel', 'couv_2_rappel', 'couv_rappel_biv', 'couv_3_rappel', 'Ensemble', 'Part des femmes (en %)', 'Part des hommes (en %)', 'Part des 0 � 24 ans de la population (en %)', 'Part des 25 � 59 ans (en %)', 'Part des 60 ans ou plus (en %)', 'dont part des 75 ans ou plus (en %)', 'Superficie', 'Densité', 'Restrictions sanitaires renforc�es']


In [127]:
#selectionner les variables d'interet
columns_list=['date', 'region_N', 'Région', 'departement', 'libelle_dep',
       "taux d'incidence", 'taux_occupation_sae', 'tx_pos',
       'tx_incid_couleur', 'taux_occupation_sae_couleur',
       'tx_pos_couleur', 'nb_orange', 'nb_rouge', 'n_dose1', 'n_complet',
       'n_rappel', 'n_2_rappel', 'n_rappel_biv', 'n_3_rappel', 'n_cum_dose1',
       'n_cum_complet', 'n_cum_rappel', 'n_cum_2_rappel', 'n_cum_rappel_biv',
       'n_cum_3_rappel', 'couv_dose1', 'couv_complet', 'couv_rappel',
       'couv_2_rappel', 'couv_rappel_biv', 'couv_3_rappel', 'Ensemble',
       'Part des femmes (en %)', 'Part des hommes (en %)',
       'Part des 0 � 24 ans de la population (en %)', 'Part des 25 � 59 ans (en %)',
       'Part des 60 ans ou plus (en %)', 'dont part des 75 ans ou plus (en %)',
       'Superficie', 'Densité',
       'Restrictions sanitaires renforc�es']
data4 = data4.select(columns_list)

In [128]:
data4.limit(5).toPandas()

,date,region_N,Région,departement,libelle_dep,taux d'incidence,taux_occupation_sae,tx_pos,tx_incid_couleur,taux_occupation_sae_couleur,tx_pos_couleur,nb_orange,nb_rouge,n_dose1,n_complet,n_rappel,n_2_rappel,n_rappel_biv,n_3_rappel,n_cum_dose1,n_cum_complet,n_cum_rappel,n_cum_2_rappel,n_cum_rappel_biv,n_cum_3_rappel,couv_dose1,couv_complet,couv_rappel,couv_2_rappel,couv_rappel_biv,couv_3_rappel,Ensemble,Part des femmes (en %),Part des hommes (en %),Part des 0 � 24 ans de la population (en %),Part des 25 � 59 ans (en %),Part des 60 ans ou plus (en %),dont part des 75 ans ou plus (en %),Superficie,Densité,Restrictions sanitaires renforc�es
0,2021-03-18,32,Hauts de France,02,Aisne,295.0300,116.4000,8.2769,rouge,rouge,orange,2,2,694,626,0,0,0,0,43506,18007,0,0,0,0,8.3000,3.4000,0.0000,0.0000,0.0000,0.0000,525.5030,"51,3","48,7","29,6","42,1","28,4","9,3","7,361.7000",0.0714,1
1,2021-03-19,32,Hauts de France,02,Aisne,307.2000,118.3000,8.3342,rouge,rouge,orange,1,2,1243,303,0,0,0,0,44749,18310,0,0,0,0,8.5000,3.5000,0.0000,0.0000,0.0000,0.0000,525.5030,"51,3","48,7","29,6","42,1","28,4","9,3","7,361.7000",0.0714,1
2,2021-03-20,32,Hauts de France,02,Aisne,322.0200,120.8000,8.5716,rouge,rouge,orange,1,2,2471,165,0,0,0,0,47220,18475,0,0,0,0,9.0000,3.5000,0.0000,0.0000,0.0000,0.0000,525.5030,"51,3","48,7","29,6","42,1","28,4","9,3","7,361.7000",0.0714,1
3,2021-03-21,32,Hauts de France,02,Aisne,321.8300,122.6000,8.5423,rouge,rouge,orange,1,2,1405,76,0,0,0,0,48625,18551,0,0,0,0,9.2000,3.5000,0.0000,0.0000,0.0000,0.0000,525.5030,"51,3","48,7","29,6","42,1","28,4","9,3","7,361.7000",0.0714,1
4,2021-03-22,32,Hauts de France,02,Aisne,346.1600,130.4000,8.8743,rouge,rouge,orange,1,2,802,497,0,0,0,0,49427,19048,0,0,0,0,9.4000,3.6000,0.0000,0.0000,0.0000,0.0000,525.5030,"51,3","48,7","29,6","42,1","28,4","9,3","7,361.7000",0.0714,1


In [129]:
print(f'Exporting Data ...')
(data4
 .write.mode('overwrite')
 .parquet(f"../Data/rawdata/final_data.parquet")
)

Exporting Data ...


In [130]:
# data 10 avril 2021
data_10_avril = data4.where(F.col('date') == '2021-04-10')

# data 12 decembre 2021
data_12_dec = data4.where(F.col('date') == '2021-12-12')

In [131]:
## export 
(data_10_avril
 .write.mode('overwrite')
 .parquet(f"../Data/rawdata/data_10_avril.parquet")
)

(data_12_dec
 .write.mode('overwrite')
 .parquet(f"../Data/rawdata/data_12_dec.parquet")
)



In [415]:
## analyse :

In [342]:
## visualisations